In [3]:
from ailabuap.io.api_caller import APICaller
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from ailabuap.transformer import (
    SimpleDeDupTransformer, 
    FuzzyDedupTransformer, 
    SimHashDedupTransformer,
    MinHashDeDupTransformer,
    SemanticDedupTransformer
)
import json
import tqdm
from data_preparation.utils.subj_sentence import TargetSentencesConcatenated
from data_preparation.utils.subj_sentence._subj_util import *


In [4]:
# path = "./data/202401&02定稿数据_追加水军_v1.0.xlsx"
# path = "./data/202403_2818brands&图像识别定稿-- 待打标水军20240408 1.xlsx"
# path = "./data/2024Q2-乔雅登品牌数据.xlsx"
# path = "./data/2024Q2-其他品牌数据-0710.xlsx"
# path = "./202407_乔雅登品牌数据_待情感打标-0814.xlsx"
# path = "./202408_乔雅登品牌数据_待情感打标-0910.xlsx"
# path = "./data/202407-08-aller_2818brands&图像识别--待跑主体情感模型-0923.xlsx"
# path = "/workspace/sentiment/2023Q4填充剂刷新情感.xlsx"
# path = "/workspace/sentiment/202409-aller_2818brands&图像识别定稿--待跑情感.xlsx"
# path = "/workspace/sentiment/aller_2818brands&图像识别定稿--202411.xlsx"
# path = "/workspace/sentiment/aller_monthly_report_2818_dae底单数据202411（含新增品牌）.xlsx"

# path = "/workspace/sentiment/data/aller_2818brands&图像识别定稿--202501-待跑情感.xlsx"

path = "/workspace/sentiment/data/202412-品牌定稿--补充小红书-未更新互动.xlsx"
path = "/workspace/sentiment/data/aller_monthly_report_2818_dae底单数据202501 (1).xlsx"
path = "/workspace/sentiment/data/202502-aller_2818brands&图像识别定稿--20250311待打标情感.xlsx"


data = pd.read_excel(path)
data.shape,data.columns

((111605, 51),
 Index(['ID', 'doc_id', 'subject1', 'subject2', 'subject3', 'quarter', 'month',
        'subject_keyword', 'pubdate', 'pubdate_full', 'pubtype', 'pubname',
        '六社媒', 'pubcode', 'author', 'src_author_id', '医美查', 'headline', 'body',
        'url', 'doc_num_of_collect', 'doc_num_of_comment', 'doc_num_of_forward',
        'doc_num_of_like', 'WOW', 'doc_num_of_read', 'fans', 'verified',
        'is_used_check', 'is_used', 'reject_word', 'valid_word',
        'effective_word', 'wa_type', 'kol_name', 'engagement_cnt',
        'influence_cnt', 'kol_name_check', 'koc_name_check', 'koc_type',
        'hospital', 'chain', 'institution_flag', 'dr_flag', 'profile_url',
        'description', 'topic1', 'topic1_score', 'topic2', 'topic2_score',
        '数据来源'],
       dtype='object'))

In [5]:
data = data[data['subject2']!="水光"]
data.shape

(96333, 51)

# 因为同一篇文章可能包含多个主体，所以虽然内容相同，但是可能需要判断情感的主体不同，所以我们要重置docid保证每行的docid都不同

In [6]:
data['origin_doc_id'] = data['doc_id']
data['doc_id'] = list(range(data.shape[0]))

In [7]:
data.columns

Index(['ID', 'doc_id', 'subject1', 'subject2', 'subject3', 'quarter', 'month',
       'subject_keyword', 'pubdate', 'pubdate_full', 'pubtype', 'pubname',
       '六社媒', 'pubcode', 'author', 'src_author_id', '医美查', 'headline', 'body',
       'url', 'doc_num_of_collect', 'doc_num_of_comment', 'doc_num_of_forward',
       'doc_num_of_like', 'WOW', 'doc_num_of_read', 'fans', 'verified',
       'is_used_check', 'is_used', 'reject_word', 'valid_word',
       'effective_word', 'wa_type', 'kol_name', 'engagement_cnt',
       'influence_cnt', 'kol_name_check', 'koc_name_check', 'koc_type',
       'hospital', 'chain', 'institution_flag', 'dr_flag', 'profile_url',
       'description', 'topic1', 'topic1_score', 'topic2', 'topic2_score',
       '数据来源', 'origin_doc_id'],
      dtype='object')

In [8]:
data.head(1).to_dict('records')

[{'ID': 53260,
  'doc_id': 0,
  'subject1': '医美产品',
  'subject2': '玻尿酸',
  'subject3': '润致',
  'quarter': '2025Q1',
  'month': 202502,
  'subject_keyword': '格格针',
  'pubdate': '20250227',
  'pubdate_full': '2025-02-27 09:49:22',
  'pubtype': '微博',
  'pubname': '新浪微博',
  '六社媒': '有',
  'pubcode': 'bg_m_sinacn',
  'author': '老朱说真话',
  'src_author_id': '2284317783',
  '医美查': nan,
  'headline': '🍶 【玉瓷面颈】项目赋能会暨广西格格针实操医师认证会2月25日📍格林美菲「格林美菲&华熙生物&润致」联合举办玉韵瓷颈·自成一格 \u200b\u200b',
  'body': '🍶 【玉瓷面颈】项目赋能会\n\n暨广西格格针实操医师认证会\n\n2月25日📍格林美菲\n\n「格林美菲&华熙生物&润致」联合举办\n\n玉韵瓷颈·自成一格',
  'url': 'http://weibo.com/2284317783/Pgc8Dgr3b',
  'doc_num_of_collect': 0,
  'doc_num_of_comment': 0,
  'doc_num_of_forward': 0,
  'doc_num_of_like': 0,
  'WOW': nan,
  'doc_num_of_read': 0,
  'fans': 11664,
  'verified': '黄V',
  'is_used_check': '是',
  'is_used': '否',
  'reject_word': nan,
  'valid_word': nan,
  'effective_word': nan,
  'wa_type': '微博',
  'kol_name': 'PGC',
  'engagement_cnt': 0,
  'influence_cnt': 0,
  'kol_na

In [9]:
data.drop_duplicates(subset=['doc_id'], inplace=True)
data['content'] = data.apply(lambda row: str(row['headline']).strip()+","+str(row['body']).strip(), axis=1)

# if "body" in data.columns:
#     data.rename(columns={"body": "content"}, inplace=True)
# data['content'] = data.apply(lambda row: str(row['headline'])+","+str(row['content']), axis=1)
data.shape, data.columns

((96333, 53),
 Index(['ID', 'doc_id', 'subject1', 'subject2', 'subject3', 'quarter', 'month',
        'subject_keyword', 'pubdate', 'pubdate_full', 'pubtype', 'pubname',
        '六社媒', 'pubcode', 'author', 'src_author_id', '医美查', 'headline', 'body',
        'url', 'doc_num_of_collect', 'doc_num_of_comment', 'doc_num_of_forward',
        'doc_num_of_like', 'WOW', 'doc_num_of_read', 'fans', 'verified',
        'is_used_check', 'is_used', 'reject_word', 'valid_word',
        'effective_word', 'wa_type', 'kol_name', 'engagement_cnt',
        'influence_cnt', 'kol_name_check', 'koc_name_check', 'koc_type',
        'hospital', 'chain', 'institution_flag', 'dr_flag', 'profile_url',
        'description', 'topic1', 'topic1_score', 'topic2', 'topic2_score',
        '数据来源', 'origin_doc_id', 'content'],
       dtype='object'))

In [10]:
processed_subject_keyword_dict = json.load(open("/workspace/sentiment/data/processed_dict_subj_medical_aesthetics_updated.json", 'r'))
processed_subject_keyword_dict.__len__()

206

In [11]:
processed_subject_keyword_dict.__len__(), processed_subject_keyword_dict['白鲸薇光']

(206, [{'id': 185, 'kws': ['*白鲸薇光*', '白鲸薇光'], 'name': '白鲸薇光'}])

In [12]:
# 添加关键词
# processed_subject_keyword_dict['安然雅'] = [{
#     "id": len(processed_subject_keyword_dict),
#     'kws': [
#         "*安然雅*",
#         "安然雅",
#         "Eroya",
#         "*Eroya*"
#     ],
#     'name': '安然雅'
# }]


# 添加缺失的主体词

In [13]:
need_add_subject_info = {
    "羽美": ["羽美"],
    "泊沃林玻尿酸": ["泊沃林"],
    "薇琳卓悦": ["卓悦"],
    "梵希思": ["梵希思", "FANXISI"],
    "希菲雅": ["希菲雅", "HYAFILIA"],
    "哈颜": ["哈颜"],
    "薇丽兰玻尿酸": ["薇丽兰"],
    "薇丽兰胶原": ["薇丽兰"],
    "新肤源": ["新肤源"],
    "修丽可铂研": ["铂研"],
    "海梦妮": ["海梦妮", "HELMONI"],
    "赛罗菲": ["赛罗菲"],
    "泊沃林胶原蛋白": ["泊沃林"],
    "薇丽兰再生": ["薇丽兰"],
    "海魅云镜": ["海魅云镜"],
    "赫伊兰": ["赫伊兰", "HERiiAN"],
    "NOVABONE": ["NOVABONE", "美骨针"],
    "励然粒子": ["励然粒子", "丽子针"]
}

In [25]:
max_num = 0
for entity, info in processed_subject_keyword_dict.items():
    # print(info)
    if int(info[0]['id']) > max_num:
        max_num = int(info[0]['id'])
    

In [23]:
for entity, re_pattern in need_add_subject_info.items():
    processed_subject_keyword_dict[entity] = [{
        "id": len(processed_subject_keyword_dict),
        "kws": [ f"*{wd}*" for wd in re_pattern] + [wd for wd in re_pattern],
        'name': 'entity'
    }]

In [ ]:
json.dump(processed_subject_keyword_dict, open("/workspace/sentiment/data/processed_dict_subj_medical_aesthetics_updated.json", 'w', encoding='utf-8'), ensure_ascii=False)

In [26]:
max_num, len(processed_subject_keyword_dict)

(205, 206)

In [27]:
# 查看是否需要补充主体
set([str(subject3).upper() for subject3 in data['subject3'].tolist()]) - set(processed_subject_keyword_dict.keys())

{'羽美 '}

In [42]:
data[data['subject3'].isin({'巴赫复欣', '露凯莉少女胶原'})].shape

(2, 45)

# 开始主体抽取

In [13]:
subj_dict_url = 'http://ess24.wisers.com/res-comm/subject-extractor-api/processtext/subj_dict'
subj_dict_url = "http://aiapi.wisers.com/subject-extractor-api-mi/processtext/subj_dict"
subj_caller = APICaller(url=subj_dict_url, method='POST', timeout=100, retry_attempt=10)


In [14]:
subj_post_data2 = []
no_subject3_list = []
for item in data.to_dict('records'):
    subjects = processed_subject_keyword_dict.get(str(item['subject3']).upper())
    if subjects is not None and item["content"] is not None and str(item["content"]).strip() != "":
        post_data =  dict(
            docid=item['doc_id'], 
            subjects=subjects,
            content=str(item["content"]), 
            is_sentence_linked=True,
        )
        subj_post_data2.append(dict(json=post_data))
    else:
        no_subject3_list.append(item['subject3'])

len(subj_post_data2), len(no_subject3_list)

(96319, 14)

In [15]:
no_subject3_list.__len__(), set(no_subject3_list).__len__()

(14, 4)

In [16]:
subj_post_data2.__len__(), subj_post_data2[1]

(96319,
 {'json': {'docid': 1,
   'subjects': [{'id': '8',
     'kws': ['*JUVÉDERM*',
      '*丰颜*',
      '*乔1*',
      '*乔2*',
      '*乔3*',
      '*乔4*',
      '*乔5*',
      '*乔亚登*',
      '*乔雅登*',
      '*乔雅登1号*',
      '*乔雅登2号*',
      '*乔雅登3号*',
      '*乔雅登4号*',
      '*乔雅登5号*',
      '*极致*',
      '*缇颜*',
      '*质颜*',
      '*雅致*',
      'JUVEDERM',
      'JUVÉDERM',
      'ULTRA',
      'ULTRAPLUS',
      'ULTRA|PLUS',
      'VOLBELLA',
      'VOLIFT',
      'VOLUMA',
      '乔雅登'],
     'name': '乔雅登'}],
   'content': '✨乔雅登朔颜广西首秀✨ 下面部精雕专家,打造「折叠度美学」新境界 💎朱格非院长领衔注射天团 📆2月15日震撼上市,开启你的颜值革命🚀 [老朱说真话的微博视频](http://t.cn/A614nB7v),✨乔雅登朔颜广西首秀✨ 下面部精雕专家,打造「折叠度美学」新境界 💎朱格非院长领衔注射天团 📆2月15日震撼上市,开启你的颜值革命🚀 [老朱说真话的微博视频](http://t.cn/A614nB7v)',
   'is_sentence_linked': True}})

In [17]:
subj_sentence_in_window = subj_caller.call_batch_async(subj_post_data2)


In [18]:
subj_sentence_in_window.success.__len__(), subj_sentence_in_window.fail.__len__()

(96006, 313)

In [19]:
subj_sentence_in_window.success[5]

{'status_code': 200,
 'result': {'docid': '167',
  'message': '',
  'hl_subjs': [],
  'ct_subjs': [{'id': '55',
    'name': '双美胶原蛋白',
    'type': '',
    'kw_idxs': [[[277, 283]], [[814, 820]]],
    'text_idxs': [[211, 369], [748, 906]]}]}}

In [20]:
subj_sentence_in_window_result = {item['result']['docid']: item['result']['ct_subjs'] for item in subj_sentence_in_window.success if item['result']['ct_subjs'].__len__() > 0}
# subj_sentence_in_window_result = {item['result']['docid']: item['result']['ct_subjs'] for item in subj_sentence_in_window.success}

subj_sentence_in_window_result.__len__()

75617

In [21]:
# 通过se的抽取结果，将包含主体的句子拼接在一起
# def combine_include_subject_sentence(data, subj_sentence_in_window_result):
#     all_docid_include_subject_sentence = []
#     for index, row in data.iterrows():
#         docid2include_subject_sentence = {}
#         docid = row['doc_id']
#         content = row['content']
#         print(content)
#         if docid not in docid2include_subject_sentence:
#             docid2include_subject_sentence["doc_id"] = docid
#             if docid in subj_sentence_in_window_result:
#                 for dic in subj_sentence_in_window_result[docid]:
#                     print(dic, content[59: 62], content[106:109], content[120:123], content[582: 585])
#                     docid2include_subject_sentence['subject'] = dic['name']
#                     docid2include_subject_sentence['sentences'] = [content[s:e] for (s, e) in dic['text_idxs']]
#         all_docid_include_subject_sentence.append(docid2include_subject_sentence)
#     print(docid2include_subject_sentence)
#     sentence_df = pd.DataFrame(docid2include_subject_sentence)
#     sentence_df['sentence_merge'] = sentence_df['sentences'].map(lambda ls: "。".join(ls))
#     return sentence_df
                    
        
    

In [22]:
# combine_include_subject_sentence(data[data['doc_id']==docid], subj_sentence_in_window_result)

# 抽取句子中的主体信息

In [23]:
def get_include_subject_subsentence_through_SE_result(deduplication_content_df, dict_subj_resp1, processed_subject_keyword_dict):
    sentences_data = []
    for i in tqdm.tqdm(deduplication_content_df.to_dict('records')):
        item = i.copy()
        # print(item)
        docid = str(item['doc_id'])
        if docid in dict_subj_resp1:
            se_res = dict(hl_subjs=[], ct_subjs=dict_subj_resp1[docid], docid=docid)
            subj_dict = subject_post_process._parse_sentiment_keywords(processed_subject_keyword_dict[str(item['subject3']).upper()])
            subj_dict = {str(k): v for k, v in subj_dict.items()}
            # 此处必须要保证输入内容和抽取时的内容拼接是相同的
            se_res_ = subject_post_process._convert_java_index(
                dict(headline="", content=str(item["content"]), se_res=se_res, docid=docid), 
                "se_res", "headline", "content"
            )
            subj_sents = subject_sentences_util.get_subj_sentences('', str(item["content"]), se_res_, subj_dict)
            item_ = item.copy()
            item_['subj_sent'] = subj_sents
            sentences_data.append(item_)
    return sentences_data

In [24]:
# 将识别到的关键词通过特殊字符mask掉
def mask_target_subject_word(sentence_output_columns, sentences_data):
    sentences = []
    for item in sentences_data:
        for ent_id, sents in item["subj_sent"].items():
            for s in sents:
                s.update({k: item[k] for k in sentence_output_columns})
                s = normalize_target(s)
                sentences.append(s)
    sentences_df = pd.DataFrame(sentences)
    print(sentences_df.shape)
    return sentences_df

In [25]:
subj_sentence_in_window_result.__len__()

75617

In [26]:
subj_sentence_in_window_result.__len__(), data.shape, 

(75617, (96333, 53))

In [27]:
sentences_data = get_include_subject_subsentence_through_SE_result(data, subj_sentence_in_window_result, processed_subject_keyword_dict)


100%|██████████| 96333/96333 [01:08<00:00, 1407.28it/s]


In [28]:
subj_sentence_in_window_result[list(subj_sentence_in_window_result.keys())[0]]

[{'id': '8',
  'name': '乔雅登',
  'type': '',
  'kw_idxs': [[[1677, 1680]], [[1794, 1797]]],
  'text_idxs': [[1648, 1754], [1754, 1891]]}]

In [29]:
sentences_data[0]

{'ID': 41079,
 'doc_id': 21,
 'subject1': '医美产品',
 'subject2': '玻尿酸',
 'subject3': '乔雅登',
 'quarter': '2025Q1',
 'month': 202502,
 'subject_keyword': '朔颜,的,下颌线,了,下颌',
 'pubdate': '20250206',
 'pubdate_full': '2025-02-06 14:56:43',
 'pubtype': '微信',
 'pubname': '微信公众号',
 '六社媒': '无',
 'pubcode': 'im_weixin',
 'author': '六一和田玲',
 'src_author_id': 'liuyihetianling_',
 '医美查': nan,
 'headline': '【文末好礼】韩女又流行上“港风”了?比裴珠泫更tvb的女人…',
 'body': '\n\n真没想到有一天能在韩国女团里,看到这样一位港风美女!!多年来一直引领亚洲审美潮流的韩系审美,最终也兜兜转转追逐上中式美学了么? \n\n黑皮、小内双、长中庭、鼻梁高挺、骨相立体清晰。 \n\n配上一头乌黑的长卷发,和大耳环,妥妥的tvb花旦脸。 \n\n175的超绝身材比例更是让他姐味十足,高级感拉满。 \n\n不仅脸港,名字更港,她就是黑厂新晋女团meovv的成员——李佳媛。 \n\n提到韩系审美,你们脑海中会浮现出什么?冷白皮、水光肌、低山根、大卧蚕…… \n\n然而,随着时代的变迁,韩系审美实际上也在悄然发生着变化。从早期的“幼态脸”到后来的“清冷美人”、“恶女风”,韩国女性对于美的追求逐渐从单一走向多元,但还未曾跳脱出对上述那些特征的偏爱。而李佳媛的出现,无疑是将这种多元化推向了一个新的高度。 \n\n李佳媛的风格在韩国娱乐圈中显得尤为独特。清晰的下颌线和立体的下颌角,展现了不同于传统韩系美女的力量感,更是让05年的她就拥有了港风美人的韵味。 \n\n近年来,韩系审美越来越注重女性的力量感,这种力量感目前大多体现在韩女对身材线条上的追求上。 \n\n大胆预测一下,这种面部的轮廓上的力量感——清晰的下颌线、立体的下颌角,不久就将会成为韩系审美的新风向。韩女也开始向着港风的不魅男、不讨好、高智感靠拢

In [30]:
sentence_output_columns = ['doc_id', 'subject3', 'pubdate' , "content", 'headline']
sentences_df = mask_target_subject_word(sentence_output_columns, sentences_data)
sentences_df.columns

(138710, 15)


Index(['ent_id', 'text', 'kws', 'is_hl', 'start_index', 'end_index', 'kw_idxs',
       'target_locs', 'doc_id', 'subject3', 'pubdate', 'content', 'headline',
       'norm_content', 'norm_target_locs'],
      dtype='object')

In [31]:
sentences_df.head()

,ent_id,text,kws,is_hl,start_index,end_index,kw_idxs,target_locs,doc_id,subject3,pubdate,content,headline,norm_content,norm_target_locs
0,8,"而如今的她,看上去更像是【有能力把自己照顾的很好】。今年,乔雅登朔颜的上市给了晨晨一个重新获...","[*juvéderm*, *豐顏*, *喬1*, *喬2*, *喬3*, *喬4*, *喬5...",False,1648,1754,"[[1677, 1680]]","[[29, 32]]",21,乔雅登,20250206,"【文末好礼】韩女又流行上“港风”了?比裴珠泫更tvb的女人…,真没想到有一天能在韩国女团里,...",【文末好礼】韩女又流行上“港风”了?比裴珠泫更tvb的女人…,"而如今的她,看上去更像是【有能力把自己照顾的很好】。今年,[unused1]朔颜的上市给了晨...","[[29, 38]]"
1,8,"比如,下巴短小、下颌后缩、面部松弛下垂等,朔颜都能帮你重塑美丽下颌线。 \n\n作为乔雅登V...","[*juvéderm*, *豐顏*, *喬1*, *喬2*, *喬3*, *喬4*, *喬5...",False,1754,1891,"[[1794, 1797]]","[[40, 43]]",21,乔雅登,20250206,"【文末好礼】韩女又流行上“港风”了?比裴珠泫更tvb的女人…,真没想到有一天能在韩国女团里,...",【文末好礼】韩女又流行上“港风”了?比裴珠泫更tvb的女人…,"比如,下巴短小、下颌后缩、面部松弛下垂等,朔颜都能帮你重塑美丽下颌线。 \n\n作为[unu...","[[40, 49]]"
2,9,"面部折叠线的塑造相当于基本框架的搭建,对于这部分的材料选择需选择支撑力特别强的再生材料(例如...","[*restylane*, *麗多*, *麗瑅*, *麗緹*, *唯瑅*, *唯緹*, *定...",False,1108,1318,"[[1229, 1231]]","[[121, 123]]",24,瑞蓝,20250204,"过年最不想面对的事……在朋友圈看到自己没P图的合照,过年根本就不敢刷朋友圈!!! \n\n年...",过年最不想面对的事……在朋友圈看到自己没P图的合照,"面部折叠线的塑造相当于基本框架的搭建,对于这部分的材料选择需选择支撑力特别强的再生材料(例如...","[[121, 130]]"
3,8,"而如今的她,看上去更像是【有能力把自己照顾的很好】。今年,乔雅登朔颜的上市给了晨晨一个重新获...","[*juvéderm*, *豐顏*, *喬1*, *喬2*, *喬3*, *喬4*, *喬5...",False,1648,1754,"[[1677, 1680]]","[[29, 32]]",27,乔雅登,20250206,"【文末好礼】韩女又流行上“港风”了?比裴珠泫更tvb的女人…,真没想到有一天能在韩国女团里,...",【文末好礼】韩女又流行上“港风”了?比裴珠泫更tvb的女人…,"而如今的她,看上去更像是【有能力把自己照顾的很好】。今年,[unused1]朔颜的上市给了晨...","[[29, 38]]"
4,8,"比如,下巴短小、下颌后缩、面部松弛下垂等,朔颜都能帮你重塑美丽下颌线。 \n\n作为乔雅登V...","[*juvéderm*, *豐顏*, *喬1*, *喬2*, *喬3*, *喬4*, *喬5...",False,1754,1891,"[[1794, 1797]]","[[40, 43]]",27,乔雅登,20250206,"【文末好礼】韩女又流行上“港风”了?比裴珠泫更tvb的女人…,真没想到有一天能在韩国女团里,...",【文末好礼】韩女又流行上“港风”了?比裴珠泫更tvb的女人…,"比如,下巴短小、下颌后缩、面部松弛下垂等,朔颜都能帮你重塑美丽下颌线。 \n\n作为[unu...","[[40, 49]]"


# 将通过句子窗口抽取到的主体句子进行拼借

In [32]:
sentences_df[sentences_df['doc_id']==5].to_dict("records")

[]

In [33]:
set(sentences_df['doc_id'].tolist()).__len__()

75617

In [34]:
dict_docid_map_sentences = {}
for docid, df_ in sentences_df.groupby(['doc_id']):
    s_list = df_.to_dict('records')
    if isinstance(docid, tuple):
        docid = docid[0]
    dict_docid_map_sentences[docid] = s_list
len(dict_docid_map_sentences)

75617

In [35]:
list(dict_docid_map_sentences.keys())[:5], sentences_df.shape

([21, 24, 27, 110, 154], (138710, 15))

In [36]:
sentences_df['sentences'] = sentences_df.apply(
    lambda x: dict_docid_map_sentences[x['doc_id']], axis=1)
sentences_df.shape

(138710, 16)

In [37]:
def concat_target_sentence_by_ent_id(input_data):
    # if input_data['媒体类型(中文)'] == 'APP':
    #     items = [i.copy() for i in input_data['sentences'] if not i['text'] in input_data['标题']]
    # else:
    #     items = [i.copy() for i in input_data['sentences']]
    items = [i.copy() for i in input_data['sentences']]

    if len(items) == 0:
        return []
    
    df_sts = pd.DataFrame(items)
    # print(df_sts)

    output = [] 
    for ent_id, df in df_sts.groupby('doc_id'):
        value = concat_target_sentences(df.to_dict('records'))
        # print(value)
        output.append(value)
    return output

In [38]:
sentence_concat_data = []
for item in tqdm.tqdm(sentences_df.to_dict('records')):
    processed_items =  concat_target_sentence_by_ent_id(item)
    sentence_concat_data.extend(processed_items)
len(sentence_concat_data)

100%|██████████| 138710/138710 [05:01<00:00, 460.33it/s]


138710

In [39]:
df_subj_content1 = pd.DataFrame(sentence_concat_data)
print(df_subj_content1.shape)
df_subj_content1 = df_subj_content1[~df_subj_content1['doc_id'].duplicated()]
df_subj_content1.shape, df_subj_content1.columns

(138710, 12)


((75617, 12),
 Index(['ent_id', 'text', 'kws', 'target_locs', 'doc_id', 'subject3', 'pubdate',
        'content', 'headline', 'num_sentence', 'norm_content',
        'norm_target_locs'],
       dtype='object'))

In [40]:
df_subj_content1.head()

,ent_id,text,kws,target_locs,doc_id,subject3,pubdate,content,headline,num_sentence,norm_content,norm_target_locs
0,8,"而如今的她,看上去更像是【有能力把自己照顾的很好】。今年,乔雅登朔颜的上市给了晨晨一个重新获...","[*juvéderm*, *豐顏*, *喬1*, *喬2*, *喬3*, *喬4*, *喬5...","[[29, 32], [146, 149]]",21,乔雅登,20250206,"【文末好礼】韩女又流行上“港风”了?比裴珠泫更tvb的女人…,真没想到有一天能在韩国女团里,...",【文末好礼】韩女又流行上“港风”了?比裴珠泫更tvb的女人…,2,"而如今的她,看上去更像是【有能力把自己照顾的很好】。今年,[unused1]朔颜的上市给了晨...","[[141, 150], [183, 192]]"
2,9,"面部折叠线的塑造相当于基本框架的搭建,对于这部分的材料选择需选择支撑力特别强的再生材料(例如...","[*restylane*, *麗多*, *麗瑅*, *麗緹*, *唯瑅*, *唯緹*, *定...","[[121, 123]]",24,瑞蓝,20250204,"过年最不想面对的事……在朋友圈看到自己没P图的合照,过年根本就不敢刷朋友圈!!! \n\n年...",过年最不想面对的事……在朋友圈看到自己没P图的合照,1,"面部折叠线的塑造相当于基本框架的搭建,对于这部分的材料选择需选择支撑力特别强的再生材料(例如...","[[338, 347]]"
3,8,"而如今的她,看上去更像是【有能力把自己照顾的很好】。今年,乔雅登朔颜的上市给了晨晨一个重新获...","[*juvéderm*, *豐顏*, *喬1*, *喬2*, *喬3*, *喬4*, *喬5...","[[29, 32], [146, 149]]",27,乔雅登,20250206,"【文末好礼】韩女又流行上“港风”了?比裴珠泫更tvb的女人…,真没想到有一天能在韩国女团里,...",【文末好礼】韩女又流行上“港风”了?比裴珠泫更tvb的女人…,2,"而如今的她,看上去更像是【有能力把自己照顾的很好】。今年,[unused1]朔颜的上市给了晨...","[[141, 150], [183, 192]]"
5,58,内外轮廓固定+眼周+口周+额头#北京少女针眉弓##眼周十全大补##天津微整形##北京微整形#...,"[*薇旖美*, 薇旖美]","[[643, 646], [654, 657], [1529, 1532], [1540, ...",110,薇旖美,2025-02-22 00:00:00,🇰🇷 全面部少女感综合打造 现在biu晚上约会妥妥出片~内外轮廓固定+眼周+口周+额头#北京...,🇰🇷 全面部少女感综合打造 现在biu晚上约会妥妥出片~内外轮廓固定+眼周+口周+额头#北京...,1,内外轮廓固定+眼周+口周+额头#北京少女针眉弓##眼周十全大补##天津微整形##北京微整形#...,"[[2410, 2419], [2427, 2436], [3308, 3317], [33..."
6,184,"今年以来华熙生物科技股份有限公司新申请注册商标 2 件,截止目前公司共持有注册商标 6930...","[*塑妍萃*, 塑妍萃, 塑然雅, *塑然雅*, sculptra, *sculptra*]","[[70, 73], [152, 155], [304, 307]]",154,塑然雅,2025-02-10 00:00:00,美力周刊 | 创健医疗重组胶原蛋白获得美国市场准入许可、华熙生物提交“光影美学”等2件商标注...,美力周刊 | 创健医疗重组胶原蛋白获得美国市场准入许可、华熙生物提交“光影美学”等2件商标注...,3,"今年以来华熙生物科技股份有限公司新申请注册商标 2 件,截止目前公司共持有注册商标 6930...","[[171, 180], [209, 218], [198, 207]]"


In [41]:
info = df_subj_content1.head(5).to_dict("records")[-3]
info

{'ent_id': '8',
 'text': '而如今的她,看上去更像是【有能力把自己照顾的很好】。今年,乔雅登朔颜的上市给了晨晨一个重新获得梦想中下颌的机会。 \n\n朔颜,这款专门针对下面部研发的玻尿酸产品,凭借它出色的支撑力和塑性能力,成为下颌轮廓新的首选。比如,下巴短小、下颌后缩、面部松弛下垂等,朔颜都能帮你重塑美丽下颌线。 \n\n作为乔雅登Vycross系列中透明质酸浓度、弹性模量和内聚力最高的产品,它能够提供强大的支撑力,能够更好的模拟骨骼的形态,承托住下面部的皮肉。 \n\n朔颜的塑形效果持久且稳定,能够立即看到明显的效果。',
 'kws': ['*juvéderm*',
  '*豐顏*',
  '*喬1*',
  '*喬2*',
  '*喬3*',
  '*喬4*',
  '*喬5*',
  '*喬亞登*',
  '*喬雅登*',
  '*喬雅登1號*',
  '*喬雅登2號*',
  '*喬雅登3號*',
  '*喬雅登4號*',
  '*喬雅登5號*',
  '*極致*',
  '*緹顏*',
  '*質顏*',
  '*雅緻*',
  'juvederm',
  'juvéderm',
  'ultra',
  'ultraplus',
  'ultra|plus',
  'volbella',
  'volift',
  'voluma',
  '喬雅登'],
 'target_locs': [[29, 32], [146, 149]],
 'doc_id': 27,
 'subject3': '乔雅登',
 'pubdate': '20250206',
 'content': '【文末好礼】韩女又流行上“港风”了?比裴珠泫更tvb的女人…,真没想到有一天能在韩国女团里,看到这样一位港风美女!!多年来一直引领亚洲审美潮流的韩系审美,最终也兜兜转转追逐上中式美学了么? \n\n黑皮、小内双、长中庭、鼻梁高挺、骨相立体清晰。 \n\n配上一头乌黑的长卷发,和大耳环,妥妥的tvb花旦脸。 \n\n175的超绝身材比例更是让他姐味十足,高级感拉满。 \n\n不仅脸港,名字更港,她就是黑厂新晋女团meovv的成员——李佳媛。 \n\n提到韩系审美,你们脑海中会浮现出什么?冷白皮、水光肌、低山根、大卧蚕…… \n\n然而,随着时代的变迁,韩系审美

In [55]:
info['text'][29:32]

'乔雅登'

# 主体情感打标

## 通过apicaller异步打标

In [43]:
# target_senti_url = 'http://ess76.wisers.com:18780/sentiment/sentence'

target_senti_url = 'http://aiapi.wisers.com/mc-target-sentiment-gpu-allergan/sentiment/sentence'
target_senti_caller = APICaller(url=target_senti_url, method='POST', timeout=100, retry_attempt=10)

In [44]:
df_subj_content1.columns, df_subj_content1.shape

(Index(['ent_id', 'text', 'kws', 'target_locs', 'doc_id', 'subject3', 'pubdate',
        'content', 'headline', 'num_sentence', 'norm_content',
        'norm_target_locs'],
       dtype='object'),
 (75617, 12))

In [45]:
set(df_subj_content1['doc_id'].to_list()).__len__(), df_subj_content1.shape

(75617, (75617, 12))

In [46]:
target_sent_request_json = [{"json": {
        "docid": row['doc_id'],
        "text": row['text'],
        "kw_idxs": row['target_locs'],
        "text_idxs": [0,len(row['text'])],
        "profile": False
    }
} for index, row in df_subj_content1.iterrows()]
target_sent_request_json.__len__(), target_sent_request_json[0]

(75617,
 {'json': {'docid': 21,
   'text': '而如今的她,看上去更像是【有能力把自己照顾的很好】。今年,乔雅登朔颜的上市给了晨晨一个重新获得梦想中下颌的机会。 \n\n朔颜,这款专门针对下面部研发的玻尿酸产品,凭借它出色的支撑力和塑性能力,成为下颌轮廓新的首选。比如,下巴短小、下颌后缩、面部松弛下垂等,朔颜都能帮你重塑美丽下颌线。 \n\n作为乔雅登Vycross系列中透明质酸浓度、弹性模量和内聚力最高的产品,它能够提供强大的支撑力,能够更好的模拟骨骼的形态,承托住下面部的皮肉。 \n\n朔颜的塑形效果持久且稳定,能够立即看到明显的效果。',
   'kw_idxs': [[29, 32], [146, 149]],
   'text_idxs': [0, 243],
   'profile': False}})

In [47]:
target_senti_resp = target_senti_caller.call_batch_async(target_sent_request_json)

In [48]:
target_senti_resp.success.__len__()

75598

In [49]:
target_senti_resp.success.__len__(), target_sent_request_json.__len__(), target_senti_resp.success[0]

(75598,
 75617,
 {'status_code': 200,
  'result': {'retCode': 'S',
   'retInfo': 'OK',
   'retData': {'score': 0.9649295210838318, 'label': 'positive'}}})

In [50]:
succ_post = [item for idx, item in enumerate(target_sent_request_json) if idx not in target_senti_resp.fail]
sentiment_tag_res = []
for post, resp in zip(succ_post, target_senti_resp.success):
    docid = post['json']['docid']
    score = resp['result']['retData']['score']
    label = resp['result']['retData']['label']
    sentiment_tag_res.append(dict(doc_id=docid, label=label, score=score))

In [51]:
data['doc_id'].to_list()[:5], pd.DataFrame(sentiment_tag_res).shape

([0, 1, 2, 3, 4], (75598, 3))

In [52]:
data_tag_res = data.merge(pd.DataFrame(sentiment_tag_res), on='doc_id', how='left')
data_tag_res.shape, data_tag_res['label'].value_counts()

((96333, 55),
 label
 neutral     43966
 positive    26674
 negative     4958
 Name: count, dtype: int64)

In [53]:
data_tag_res[data_tag_res['label'].isna()].shape

(20735, 55)

In [54]:
data_tag_res.to_excel("./data/202502-aller_2818brands&图像识别定稿--20250311打标结果.xlsx", index=False)

## 通过一条一条循环遍历去调用

In [45]:
sentiment_tag_res.__len__(), fail_sentiment_tag_res.__len__()

(109859, 0)

In [46]:
data_tag_res = data.merge(pd.DataFrame(sentiment_tag_res), on='doc_id', how='left')
data_tag_res.shape, data_tag_res['label'].value_counts()

((144204, 43),
 label
 neutral     71137
 positive    33215
 negative     5507
 Name: count, dtype: int64)

In [47]:
data_tag_res.head()

,doc_id,subject1,subject2,subject3,pubdate,quarter,pubcode,month,pubtype,pubname,...,topic2_score,数据来源,六社媒,账号水军 - 1是0否,内容水军 - 1是0否,内容水军概率值,origin_doc_id,content,score,label
0,0,医美产品,玻尿酸,乔雅登,20240718,2024Q3,wm_xiaohongshucn_app,202407,APP,小红书 APP,...,0.044223,2818表,有,0.0,0.0,0.005851,fec06802990949e3459dd558c492f317,"QYD为什么那么贵,用过的都知道,虽然相比其他BNS会贵,但物有所值。同样是BNS,为啥选Q...",0.990412,neutral
1,1,医美产品,玻尿酸,乔雅登,20240703,2024Q3,wm_xiaohongshucn_app,202407,APP,小红书 APP,...,0.185659,2818表,有,0.0,0.0,0.000080,38b00e662e174a3e25c80a6f5ed87639,"国际美学导师👨‍⚕王晔博士7月亲诊日,👨‍⚕王晔博士7月坐诊限量预约中 医学博士/国际高级审...",0.986113,neutral
2,2,医美产品,玻尿酸,乔雅登,20240710,2024Q3,wm_xiaohongshucn_app,202407,APP,小红书 APP,...,0.000934,2818表,有,0.0,0.0,0.000975,a3120f74c6633708b51c732967504dd5,"评委日记 |口周美学动态情绪美学,关于口周美学动态情绪美学,作为菁英医生,优雅艺术家评委,受...",0.976688,neutral
3,3,医美产品,玻尿酸,乔雅登,20240703,2024Q3,im_weixin,202407,微信,微信公众号,...,0.117871,2818表,无,0.0,0.0,0.000040,f6b4494ccb9057290b999ed3cc949104,"乔雅登缇颜科普:技术介绍、产品优势和操作方法,乔雅登·缇颜,动态填充,弹性极佳,适中硬度和内...",0.957406,positive
4,4,医美产品,玻尿酸,乔雅登,20240729,2024Q3,im_weixin,202407,微信,微信公众号,...,0.168105,2818表,无,0.0,0.0,0.075176,e6bc25136476feef9a6f2b38859b9e9b,"授课专家简介1:杭州整形医院 杨伟渊,杨伟渊 \n\n 副主任医师 \n\n 中...",0.990802,neutral


In [52]:
data_tag_res['label'].value_counts(), data_tag_res[data_tag_res['label'].isna()].shape

(label
 neutral     71137
 positive    33215
 negative     5507
 Name: count, dtype: int64,
 (34345, 43))

In [50]:
# # 
import re
docid2no_include = []
for js in test_json:
    content = str(js['headline']) + "," + str(js['body'])
    if js['subject3'].upper() in processed_subject_keyword_dict:
        keywords = processed_subject_keyword_dict[js['subject3'].upper()]
        print(js['doc_id'], keywords, content)
        sign_list = []
        for word in keywords[0]['kws']:
            # print(re.findall('[\u4e00-\u9fa5]+', word))
            for wd in re.findall('[\u4e00-\u9fa5]+', word):
                sign_list.append(wd in content)
        docid2no_include.append({"doc_id": js['doc_id'],  "sign":  any(sign_list)})
        

NameError: name 'test_json' is not defined

In [ ]:
pd.DataFrame(docid2no_include)['sign'].value_counts()

# 使用apicaller并行工具包调用，未测试

In [77]:
label_map = {
    'negative': -1, 
    'neutral': 0, 
    'positive': 1
}


In [ ]:
senti_resp  = target_senti_caller.call_batch_async(senti_post_data, progress=True)

In [ ]:
succ_post = [item['json'] for idx, item in enumerate(senti_post_data) if idx not in senti_resp.fail ]
succ_post.__len__()

In [ ]:
dict_senti_res = {}
for post, resp in zip(succ_post, senti_resp.success):
    iid = post['id']
    result = resp['result']['retData']['label']
    dict_senti_res[iid] = label_map[result]

In [ ]:
sentences_df['sentiment'] = sentences_df['index'].apply(lambda x: dict_senti_res[x] if x in dict_senti_res else None)
sentences_df['sentiment'].value_counts()

# 将正负中三种情感整合成负面以及非负两类

In [84]:
def get_combine_sentiment_tag(data):
    content_sentiment_tag_res = {}
    for doc_id, group_df in data.groupby("doc_id"):
        multi_sentence_sentiment = set(group_df['label'].tolist())
        if group_df.shape[0] > 5:
            print(doc_id, multi_sentence_sentiment)
        if "negative" in multi_sentence_sentiment:
            content_sentiment_tag_res[doc_id] = "负面"
        elif "positive" in multi_sentence_sentiment or "neutral" in multi_sentence_sentiment:
            content_sentiment_tag_res[doc_id] = "非负"
        else:
            content_sentiment_tag_res[doc_id] = None
    return content_sentiment_tag_res

In [85]:
docid2sentiment = get_combine_sentiment_tag(data_tag_res)

In [86]:
data_tag_res['AI_sentiment_tag'] = data_tag_res['doc_id'].map(lambda x: docid2sentiment[x] if x in docid2sentiment else None)

In [87]:
import warnings
warnings.filterwarnings("ignore")
data_tag_res.to_excel("艾尔建202403打标结果.xlsx", index=False)

In [88]:
data_tag_res[data_tag_res['AI_sentiment_tag'].isna()].shape

(20761, 35)

In [89]:
data_tag_res.shape

(84599, 35)

In [93]:
data_tag_res[data_tag_res['origin_doc_id'] == "5857035c2b63701b0eec880a58eea1c1"].head()

,doc_id,subject1,subject2,subject3,pubdate,quarter,pubcode,month,pubtype,pubname,...,influence_cnt,koc_type_check,uid,body,pubdate_full,origin_doc_id,content,score,label,AI_sentiment_tag
1,1,医美产品,玻尿酸,菲洛嘉,2024-03-14,2024Q1,bg_m_sinacn,202403,C,新浪微博,...,17,医生,NaN,@FILLMED菲欧曼医学 #fillmed菲欧曼# [上海·思南公馆](http://t....,2024-03-14 18:15:41,5857035c2b63701b0eec880a58eea1c1,"在《繁花》的世界里,时间是一位细腻的画家,以无数的色彩和线条,勾勒出生命的厚重与轻盈。菲欧曼...",0.899018,positive,非负
